# Part 9.1: Multi-GPU & Distributed Training Fundamentals

## Scaling Beyond Single GPU

Throughout this course, we've optimized training on a single GPU using techniques like LoRA, QLoRA, and Unsloth. Now we'll learn how to **scale training across multiple GPUs** — essential for larger models and faster iteration.

**Learning Objectives:**
1. Understand why and when to distribute training
2. Learn the three main parallelism strategies
3. Understand DeepSpeed ZeRO stages
4. Compare FSDP vs DeepSpeed
5. Get introduced to HuggingFace Accelerate

---

## 1. Why Distribute Training?

When training on a single GPU, you eventually hit one of two walls:

### 1.1 The Two Problems

**Problem 1: Memory Limit — Model Doesn't Fit**

GPU memory must hold:
- Model parameters
- Optimizer states (Adam has 2x parameters)
- Gradients
- Activations (for backprop)

For a 7B parameter model in FP32:
```
Parameters:      7B × 4 bytes = 28 GB
Optimizer (Adam): 7B × 8 bytes = 56 GB
Gradients:       7B × 4 bytes = 28 GB
─────────────────────────────────────
Total:                         112 GB (without activations!)
```

Even an A100 (80GB) can't fit this!

**Problem 2: Speed Limit — Training is Too Slow**

Even if the model fits, you might want:
- Faster experimentation cycles
- Larger batch sizes for better convergence
- Quicker hyperparameter searches

### 1.2 What Distribution Solves

| Problem | Solution | How It Helps |
|---------|----------|-------------|
| Model too large | Shard model across GPUs | Each GPU holds fraction of memory |
| Training too slow | Process batches in parallel | N GPUs ≈ N× throughput |
| Batch size limited | Accumulate across GPUs | Effective batch = N × local batch |

### 1.3 Your Context

You've been training on:
- **Colab T4**: 16GB VRAM, single GPU
- **Models**: 0.5B - 3B parameters
- **Techniques**: QLoRA, Unsloth optimizations

With **Lightning.ai's 2 GPUs**, you can:
- Train ~2× faster with data parallelism
- Fit larger models with memory sharding
- Use larger batch sizes for better gradients

---

## 2. The Three Parallelism Strategies

There are three fundamental ways to distribute training across GPUs.

1. Data Parallelism
2. Model/Tensor Parallelism
3. Fully Sharded Data Parallelism -> FSDP Zero

### 2.1 Data Parallelism (DP)

**Concept:** Each GPU has a complete copy of the model, but processes different data.

```
                    Training Data
                         │
              ┌──────────┴──────────┐
              ▼                     ▼
         Batch 1               Batch 2
              │                     │
              ▼                     ▼
    ┌─────────────────┐   ┌─────────────────┐
    │     GPU 0       │   │     GPU 1       │
    │  Full Model     │   │  Full Model     │
    │    Copy         │   │    Copy         │
    └────────┬────────┘   └────────┬────────┘
             │                     │
             ▼                     ▼
        Gradients 1           Gradients 2
             │                     │
             └──────────┬──────────┘
                        ▼
              All-Reduce (Average)
                        │
                        ▼
                 Update Weights
                 (synchronized)
```

**Pros:**
- Simple to implement
- Near-linear speedup with more GPUs
- No code changes to model

**Cons:**
- Model must fit on each GPU
- Memory not reduced (actually slightly increased due to gradient buffers)

**Best for:** Models that fit on one GPU, but you want faster training.

### 2.2 Model Parallelism (MP) / Tensor Parallelism (TP)

**Concept:** Split the model itself across GPUs.

```
                    Input
                      │
                      ▼
    ┌─────────────────────────────────┐
    │            GPU 0                │
    │       Layers 1 - 12             │
    └────────────────┬────────────────┘
                     │ activations
                     ▼
    ┌─────────────────────────────────┐
    │            GPU 1                │
    │       Layers 13 - 24            │
    └────────────────┬────────────────┘
                     │
                     ▼
                  Output
```

**Pros:**
- Can train models larger than single GPU memory
- Each GPU only holds part of model

**Cons:**
- Complex to implement
- GPUs wait for each other (pipeline bubbles)
- Communication overhead between GPUs
- Doesn't speed up training much

**Best for:** Very large models that can't fit any other way.

### 2.3 Fully Sharded Data Parallelism (FSDP / ZeRO)

**Concept:** Shard parameters, gradients, and optimizer states across GPUs. Gather when needed.

```
              Before: Data Parallelism
    ┌─────────────────┐   ┌─────────────────┐
    │     GPU 0       │   │     GPU 1       │
    │ Full Params     │   │ Full Params     │
    │ Full Optimizer  │   │ Full Optimizer  │
    │ Full Gradients  │   │ Full Gradients  │
    └─────────────────┘   └─────────────────┘
         Memory: 100%          Memory: 100%


               After: FSDP / ZeRO-3
    ┌─────────────────┐   ┌─────────────────┐
    │     GPU 0       │   │     GPU 1       │
    │ Params Shard 1  │   │ Params Shard 2  │
    │ Optim Shard 1   │   │ Optim Shard 2   │
    │ Grads Shard 1   │   │ Grads Shard 2   │
    └─────────────────┘   └─────────────────┘
         Memory: ~50%          Memory: ~50%


    During Forward/Backward:
    - All-gather params when needed
    - Compute
    - Reduce-scatter gradients
    - Discard non-local params
```

**Pros:**
- Massive memory reduction (scales with GPU count)
- Can train very large models
- Combines benefits of DP and MP

**Cons:**
- More communication overhead
- Slightly more complex setup

**Best for:** Training large models efficiently across multiple GPUs.

### 2.4 Comparison Summary

| Strategy | Memory Reduction | Speedup | Complexity | Use Case |
|----------|-----------------|---------|------------|----------|
| **Data Parallel** | None | ~N× | Low | Model fits, want speed |
| **Model Parallel** | ~N× | Minimal | High | Huge models only |
| **FSDP / ZeRO** | ~N× | Good | Medium | Large models + speed |

---

## 3. DeepSpeed ZeRO Stages

DeepSpeed's ZeRO (Zero Redundancy Optimizer) has three stages, each sharding more aggressively.

### 3.1 What Gets Sharded at Each Stage

```
┌─────────────────────────────────────────────────────────────────┐
│                    GPU MEMORY BREAKDOWN                         │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   ┌─────────────┐                                               │
│   │ Activations │  ← Stored for backward pass                   │
│   ├─────────────┤                                               │
│   │  Gradients  │  ← ZeRO-2+ shards these                       │
│   ├─────────────┤                                               │
│   │  Optimizer  │  ← ZeRO-1+ shards these (largest!)            │
│   │   States    │                                               │
│   ├─────────────┤                                               │
│   │ Parameters  │  ← ZeRO-3 shards these                        │
│   └─────────────┘                                               │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### 3.2 ZeRO Stage Comparison

| Stage | What's Sharded | Memory Savings | Communication | Speed |
|-------|---------------|----------------|---------------|-------|
| **ZeRO-1** | Optimizer states | ~4× | Low | Fast |
| **ZeRO-2** | + Gradients | ~8× | Medium | Good |
| **ZeRO-3** | + Parameters | ~N× (linear with GPUs) | High | Slower |

**Memory formula (approximate):**
```
Standard DP:  Model × (1 + 1 + optimizer_multiplier) per GPU
ZeRO-1:       Model × (1 + 1 + optimizer_multiplier/N) per GPU
ZeRO-2:       Model × (1 + (1 + optimizer_multiplier)/N) per GPU
ZeRO-3:       Model × (1 + 1 + optimizer_multiplier)/N per GPU
```

### 3.3 When to Use Each Stage

**ZeRO-1:**
- Model fits on GPU with some room
- Want memory savings with minimal overhead
- Good default starting point

**ZeRO-2:**
- Model barely fits on GPU
- Need more memory for larger batches
- Acceptable communication overhead

**ZeRO-3:**
- Model doesn't fit on single GPU at all
- Maximum memory efficiency needed
- Willing to trade speed for capability

### 3.4 Visual: Memory Reduction with ZeRO

For a 7B parameter model with 2 GPUs:

```
Standard Data Parallel (per GPU):
├─────────────────────────────────────────────────────┤ 112 GB
│ Params (28GB) │ Optimizer (56GB) │ Gradients (28GB) │
└─────────────────────────────────────────────────────┘

ZeRO-1 (per GPU):
├─────────────────────────────────────────┤ 84 GB
│ Params (28GB) │ Optim (28GB) │ Grads (28GB) │
└─────────────────────────────────────────┘

ZeRO-2 (per GPU):
├────────────────────────────────┤ 56 GB
│ Params (28GB) │ Optim+Grads (28GB) │
└────────────────────────────────┘

ZeRO-3 (per GPU):
├──────────────────┤ 28 GB
│ Everything sharded │
└──────────────────┘
```

---

## 4. FSDP vs DeepSpeed

Both achieve similar goals but come from different ecosystems.

### 4.1 Comparison

| Aspect | DeepSpeed | FSDP |
|--------|-----------|------|
| **Origin** | Microsoft | Meta (PyTorch native) |
| **Integration** | Requires DeepSpeed library | Built into PyTorch |
| **Configuration** | JSON config file | Python API |
| **Features** | ZeRO-1/2/3, Offload, Infinity | Sharding, Mixed Precision |
| **Maturity** | More features, well-tested | Newer, rapidly improving |
| **HF Support** | Excellent (Trainer + Accelerate) | Good (Trainer + Accelerate) |
| **CPU Offload** | Yes (ZeRO-Offload) | Yes (recent versions) |

### 4.2 When to Choose

**Choose DeepSpeed when:**
- Need maximum memory efficiency
- Want CPU/NVMe offloading
- Using HuggingFace Trainer (great integration)
- Need proven, battle-tested solution

**Choose FSDP when:**
- Want PyTorch-native solution
- Prefer Python configuration over JSON
- Already using PyTorch ecosystem heavily
- Want simpler debugging (native PyTorch)

---

## 5. HuggingFace Accelerate

Accelerate is a library that simplifies distributed training by providing a unified interface.

### 5.1 What Accelerate Does

```
┌─────────────────────────────────────────────────────────────┐
│                    Your Training Code                       │
└─────────────────────────────┬───────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                   HuggingFace Accelerate                    │
│                                                             │
│   - Handles device placement                                │
│   - Manages distributed communication                       │
│   - Wraps model, optimizer, dataloader                      │
│   - Provides unified API                                    │
└─────────────────────────────┬───────────────────────────────┘
                              │
         ┌────────────────────┼────────────────────┐
         ▼                    ▼                    ▼
┌─────────────────┐  ┌─────────────────┐  ┌─────────────────┐
│   Single GPU    │  │   DeepSpeed     │  │     FSDP        │
└─────────────────┘  └─────────────────┘  └─────────────────┘
```

**Key benefit:** Write code once, run anywhere (single GPU, multi-GPU, multi-node).

### 5.2 Minimal Code Changes

**Before (Single GPU):**
```python
model = MyModel()
optimizer = torch.optim.AdamW(model.parameters())
dataloader = DataLoader(dataset, batch_size=8)

model.to('cuda')
for batch in dataloader:
    batch = batch.to('cuda')
    outputs = model(batch)
    loss = compute_loss(outputs)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
```

**After (Multi-GPU with Accelerate):**
```python
from accelerate import Accelerator

accelerator = Accelerator()

model = MyModel()
optimizer = torch.optim.AdamW(model.parameters())
dataloader = DataLoader(dataset, batch_size=8)

# Accelerate handles everything!
model, optimizer, dataloader = accelerator.prepare(
    model, optimizer, dataloader
)

for batch in dataloader:
    outputs = model(batch)  # No .to('cuda') needed!
    loss = compute_loss(outputs)
    accelerator.backward(loss)  # Instead of loss.backward()
    optimizer.step()
    optimizer.zero_grad()
```

**Changes:**
1. Create `Accelerator()`
2. Call `accelerator.prepare()` on model, optimizer, dataloader
3. Use `accelerator.backward()` instead of `loss.backward()`
4. Remove manual `.to('cuda')` calls

### 5.3 Configuration with `accelerate config`

Before running, you configure Accelerate:

```bash
$ accelerate config

In which compute environment are you running? multi-GPU
How many machines? 1
How many GPUs? 2
Do you want to use DeepSpeed? yes
Which ZeRO stage? 2
...
```

This creates a config file. Then launch with:

```bash
$ accelerate launch train.py
```

---

## 6. Practical Considerations

### 6.1 Effective Batch Size

With distributed training, your effective batch size changes:

```
Effective Batch Size = per_gpu_batch × num_gpus × gradient_accumulation_steps
```

**Example:**
- per_gpu_batch = 4
- num_gpus = 2
- gradient_accumulation = 8
- **Effective batch = 4 × 2 × 8 = 64**

**Why it matters:**
- Learning rate often needs adjustment with batch size
- Common rule: scale LR linearly with batch size
- Or use learning rate warmup

### 6.2 Communication Overhead

Distributed training adds communication costs:

| Operation | When | Cost |
|-----------|------|------|
| All-Reduce | Gradient sync | O(model_size) |
| All-Gather | ZeRO-3 forward | O(model_size) |
| Reduce-Scatter | ZeRO gradient sync | O(model_size) |

**Tips to reduce overhead:**
- Use gradient accumulation (fewer syncs)
- Overlap communication with computation
- Use fast interconnects (NVLink > PCIe)

### 6.3 Common Issues

| Issue | Cause | Solution |
|-------|-------|----------|
| OOM on one GPU | Uneven model distribution | Check sharding config |
| Training hangs | Deadlock in communication | Check all processes reach sync points |
| Loss is NaN | Gradient explosion | Lower LR, add gradient clipping |
| Slow training | Too much communication | Increase batch size, use ZeRO-1/2 |
| Different results | Random seed not synced | Set seed on all processes |

---

## 7. Summary: Choosing Your Strategy

### Decision Tree

```
Does your model fit on a single GPU?
│
├─ YES → Do you want faster training?
│        │
│        ├─ YES → Use Data Parallelism (DDP) or ZeRO-1
│        │
│        └─ NO → Stay with single GPU (simpler)
│
└─ NO → How much memory do you need?
         │
         ├─ A little more → ZeRO-2
         │
         └─ Much more → ZeRO-3 or FSDP
```

### For Your Lightning.ai Setup (2 GPUs)

**Recommended approach:**
1. Start with **Accelerate + DeepSpeed ZeRO-2**
2. Use HuggingFace Trainer for simplicity
3. Monitor memory usage
4. Upgrade to ZeRO-3 if needed

---

## 8. Key Takeaways

1. **Data Parallelism** — Same model on each GPU, different data. Simple, fast, no memory savings.

2. **Model Parallelism** — Split model across GPUs. Complex, for huge models only.

3. **FSDP / ZeRO** — Best of both worlds. Shard everything, gather when needed.

4. **ZeRO Stages:**
   - ZeRO-1: Shard optimizer → 4× memory reduction
   - ZeRO-2: + Shard gradients → 8× reduction
   - ZeRO-3: + Shard parameters → N× reduction

5. **Accelerate** — Unified interface for distributed training. Minimal code changes.

6. **Effective batch size** — Changes with distribution. Adjust learning rate accordingly.

---

## Next Steps

In **Part 9.2**, we'll:
- Set up Lightning.ai environment
- Configure Accelerate with DeepSpeed
- Run distributed fine-tuning on 2 GPUs
- Compare performance vs single GPU

Get your Lightning.ai account ready!

---

## References

- [DeepSpeed Documentation](https://www.deepspeed.ai/)
- [PyTorch FSDP Tutorial](https://pytorch.org/tutorials/intermediate/FSDP_tutorial.html)
- [HuggingFace Accelerate](https://huggingface.co/docs/accelerate/)
- [ZeRO Paper](https://arxiv.org/abs/1910.02054)
- [DeepSpeed ZeRO Tutorial](https://www.deepspeed.ai/tutorials/zero/)